In [248]:
import json
import urllib
import os
import re

In [249]:
def get_initial_board_from_local(local_file_path = 'm8n2.txt'):
    
    file1 = open(local_file_path, 'r')

    Lines = file1.readlines()

    board_id = 0 
    
    aggregate_board = []
    
    for row, line in enumerate(Lines):

        # getting lines with the initial positions
        if "/" in line:

            # getting lines start with the white
            if line.split()[1] == 'w':
                
                # empty board as a current board
                current_board = {}
                
                # adding id to the board, starting with 0
                current_board['id'] = board_id         
                board_id += 1
                
                # adding raw board position to the board
                raw_board_position = line.split()[0].split('/')       
                current_board['board'] = raw_board_position
                
                # adding moves to the board
                next_line = Lines[row + 1]
                move = next_line.replace('\n', '')
                move = re.sub('[0-9].', '', move)
                move = move.split()
                current_board['moves'] = move
                
                # appending the current board to aggregate board
                aggregate_board.append(current_board)
    
    return aggregate_board

In [250]:
def get_initial_board_from_web(url = "https://wtharvey.com/m8n2.txt"):
    
    
    file = urllib. request. urlopen(url)
    
    Lines = []
    
    for line in file:
        decoded_line = line. decode("utf-8")
        Lines.append(decoded_line)

    board_id = 0 
    
    aggregate_board = []
    
    for row, line in enumerate(Lines):

        # getting lines with the initial positions
        if "/" in line:

            # getting lines start with the white
            if line.split()[1] == 'w':
                
                # empty board as a current board
                current_board = {}
                
                # adding id to the board, starting with 0
                current_board['id'] = board_id         
                board_id += 1
                
                # adding raw board position to the board
                raw_board_position = line.split()[0].split('/')       
                current_board['board'] = raw_board_position
                
                # adding moves to the board
                next_line = Lines[row + 1]
                move = next_line.replace('\n', '')
                move = re.sub('[0-9].', '', move)
                move = move.split()
                current_board['moves'] = move
                
                # appending the current board to aggregate board
                aggregate_board.append(current_board)
    
    return aggregate_board

In [251]:
def get_transform_board(board):
    
    # create a mapping 
    mapping = {'r': {'type':'rook', 'side': 'white'},
           'b': {'type':'bishop', 'side': 'white'},
           'n': {'type': 'knight', 'side': 'white'},
           'k': {'type': 'king', 'side': 'white'},
           'q': {'type': 'queen', 'side': 'white'},
           'p': {'type': 'pawn', 'side': 'white'},
           'R': {'type':'rook', 'side': 'black'},
           'B': {'type':'bishop', 'side': 'black'},
           'N': {'type': 'knight', 'side': 'black'},
           'K': {'type': 'king', 'side': 'black'},
           'Q': {'type': 'queen', 'side': 'black'},
           'P': {'type': 'pawn', 'side': 'black'},
           '0': {None}}
    
    
    new_board = []
    
    for row in board:
        # transform the row data format to chessy initial board format 
        #(for example, 'r2qkb1r' --> ['r', '0', '0', 'q', 'k', 'b', '0', 'r'])
        transform_row = []
        for item in row:
            if item.isalpha():
                transform_row.append(item)
            else:
                for element in int(item) * ['0']:
                    transform_row.append(element)
                    
        # map alphas to chessy's data input type (for example, from 'r' to {'type':'rook', 'side': 'white'})
        transform_row = [mapping[item] for item in transform_row]
        new_board.append(transform_row)
    
    return new_board
        
        
    
    

In [252]:
def get_aggregate_transform_board(aggregate_board):
    
    for index, current_block in enumerate(aggregate_board):
        raw_board = current_block['board']
        new_board = get_transform_board(raw_board)
        current_block['board'] = new_board
    
    aggregate_transform_board = aggregate_board
    
    return aggregate_transform_board

In [253]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError


In [254]:
def save_json_files(aggregate_transform_board):

    path = os. getcwd() + '/puzzle_folder'

    if not os.path.exists(path):
        os.makedirs(path)
    
    for index, board in enumerate(aggregate_transform_board):
        
        json_board = json.dumps(board, default=set_default)
        
        board_name = path + '/' + str(board['id']) + '_board.json'
        
        jsonFile = open(board_name, "w")
        jsonFile.write(json_board)
        jsonFile.close()
    

In [263]:
aggregate_board = get_initial_board_from_web()

In [256]:
aggregate_transform_board = get_aggregate_transform_board(aggregate_board)

In [257]:
save_json_files(aggregate_transform_board)

### Testing:

1. Testing on get_initial_board_from_web function:

In [268]:
print(aggregate_board[1])

{'id': 1, 'board': ['1rb4r', 'pkPp3p', '1b1P3n', '1Q6', 'N3Pp2', '8', 'P1P3PP', '7K'], 'moves': ['Qd', 'Ka', 'cxbN#']}


In [270]:
print(aggregate_board[10])

{'id': 10, 'board': ['r2q1b1r', '1pN1n1pp', 'p1n3k1', '4Pb2', '2BP4', '8', 'PPP3PP', 'R1BQ1RK1'], 'moves': ['Qg', 'Bxg', 'Bf']}


In [272]:
aggregate_board[1]

{'id': 1,
 'board': ['1rb4r', 'pkPp3p', '1b1P3n', '1Q6', 'N3Pp2', '8', 'P1P3PP', '7K'],
 'moves': ['Qd', 'Ka', 'cxbN#']}

In [271]:
aggregate_board[10]


{'id': 10,
 'board': ['r2q1b1r',
  '1pN1n1pp',
  'p1n3k1',
  '4Pb2',
  '2BP4',
  '8',
  'PPP3PP',
  'R1BQ1RK1'],
 'moves': ['Qg', 'Bxg', 'Bf']}

2. Testing on get_aggregate_transform_board function:

In [273]:
aggregate_transform_board[1]

{'id': 1,
 'board': [[{None},
   {'type': 'rook', 'side': 'white'},
   {'type': 'bishop', 'side': 'white'},
   {None},
   {None},
   {None},
   {None},
   {'type': 'rook', 'side': 'white'}],
  [{'type': 'pawn', 'side': 'white'},
   {'type': 'king', 'side': 'white'},
   {'type': 'pawn', 'side': 'black'},
   {'type': 'pawn', 'side': 'white'},
   {None},
   {None},
   {None},
   {'type': 'pawn', 'side': 'white'}],
  [{None},
   {'type': 'bishop', 'side': 'white'},
   {None},
   {'type': 'pawn', 'side': 'black'},
   {None},
   {None},
   {None},
   {'type': 'knight', 'side': 'white'}],
  [{None},
   {'type': 'queen', 'side': 'black'},
   {None},
   {None},
   {None},
   {None},
   {None},
   {None}],
  [{'type': 'knight', 'side': 'black'},
   {None},
   {None},
   {None},
   {'type': 'pawn', 'side': 'black'},
   {'type': 'pawn', 'side': 'white'},
   {None},
   {None}],
  [{None}, {None}, {None}, {None}, {None}, {None}, {None}, {None}],
  [{'type': 'pawn', 'side': 'black'},
   {None},
   {'

3. Testing on save_json_files function

In [240]:
path = os. getcwd() + '/puzzle_folder/1_board.json'

with open(path, 'r') as j:
     contents = json.loads(j.read())

print(contents)

print(contents['id'])

{'id': 1, 'board': [[[None], {'type': 'rook', 'side': 'white'}, {'type': 'bishop', 'side': 'white'}, [None], [None], [None], [None], {'type': 'rook', 'side': 'white'}], [{'type': 'pawn', 'side': 'white'}, {'type': 'king', 'side': 'white'}, {'type': 'pawn', 'side': 'black'}, {'type': 'pawn', 'side': 'white'}, [None], [None], [None], {'type': 'pawn', 'side': 'white'}], [[None], {'type': 'bishop', 'side': 'white'}, [None], {'type': 'pawn', 'side': 'black'}, [None], [None], [None], {'type': 'knight', 'side': 'white'}], [[None], {'type': 'queen', 'side': 'black'}, [None], [None], [None], [None], [None], [None]], [{'type': 'knight', 'side': 'black'}, [None], [None], [None], {'type': 'pawn', 'side': 'black'}, {'type': 'pawn', 'side': 'white'}, [None], [None]], [[None], [None], [None], [None], [None], [None], [None], [None]], [{'type': 'pawn', 'side': 'black'}, [None], {'type': 'pawn', 'side': 'black'}, [None], [None], [None], {'type': 'pawn', 'side': 'black'}, {'type': 'pawn', 'side': 'black'

1. the naming of the functions / variables (like ffn board, etc, not initial board) - 

2. use pytest (consider pytest mock function to replace url.request.read )

3. we can make tests static (specifically read one file and test the first row)

4. we should write docstring / explaination of the function, mostly input format / output format are importamt

5. replace all the testing ranges with specific examples (all the testing ranges are testing whether the initial file was right. which can be written in the origianl py file but not on the test file), in the test file we want to test the functions 

6. read about unit test / functional test

7. read about cli function in python

1. reduce the duplicate lines on 2,3,4 moves

2. set up a flag for user to specify where to save the files

3. maybe combine the files to 1, seperate files (not super necessary)

4. limit the number of puzzles to generate, whether you want to randomize

5. limit the moves to only be 2, 3, 4 (enum)

6. python command.py --number_of_moves='' --> directly run without python command.py

